### LCEL -LangChain Expression Language

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [2]:
#initializing env variable
load_dotenv(dotenv_path='D:\Project\.env.txt')

True

#### Piping prompt, Model and output parser

In [3]:
list_instructions= CommaSeparatedListOutputParser().get_format_instructions()

In [40]:
chat=ChatOpenAI(model_name='gpt-4',model_kwargs={'seed':365},temperature=0,max_tokens=500)

D:\Software\Anaconda\envs\langchain_env\lib\site-packages\IPython\core\interactiveshell.py:3517: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [5]:
chat_template=ChatPromptTemplate.from_messages([
                                ('human',"Suggest me {pet} names?\n"+list_instructions)])

In [6]:
list_output_parser=CommaSeparatedListOutputParser()

In [7]:
#traditional method
chat_template_result=chat_template.invoke({'pet':'dog'})
chat_result=chat.invoke(chat_template_result)
result=list_output_parser.invoke(chat_result)
result

['Max',
 'Bella',
 'Charlie',
 'Lucy',
 'Cooper',
 'Daisy',
 'Buddy',
 'Lola',
 'Rocky',
 'Sadie',
 'Zeus',
 'Molly',
 'Jack',
 'Bailey',
 'Toby',
 'Stella',
 'Duke',
 'Roxy',
 'Oliver',
 'Luna']

In [8]:
#LCEL
chain=chat_template | chat| list_output_parser
chain.invoke({'pet':'dog'})

['Max',
 'Bella',
 'Charlie',
 'Lucy',
 'Cooper',
 'Daisy',
 'Buddy',
 'Lola',
 'Rocky',
 'Sadie',
 'Zeus',
 'Molly',
 'Jack',
 'Bailey',
 'Toby',
 'Stella',
 'Duke',
 'Roxy',
 'Oliver',
 'Luna']

#### Batch

In [9]:
chat_template=ChatPromptTemplate.from_messages([
                                ('human',"Suggest me how to train my {pet}?\n"+list_instructions)])

In [10]:
#LCEL
chain=chat_template | chat

In [11]:
#Batch
chain.batch([{'pet':'dog'},{'pet':'dragon'}])

[AIMessage(content='Start with basic commands, Use positive reinforcement, Be consistent with commands, Train in various places, Use high-value treats, Keep training sessions short, Be patient, Avoid punishment, Socialize your dog, Gradually increase the difficulty level, Practice commands in different situations, Reward good behavior immediately, Use a leash during training, Keep training fun and engaging, Involve all family members in training, Hire a professional trainer if needed', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 44, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5359b8ef-a200-49eb-9ca4-99b033c2cb76-0', usage_metadata=

#### Streaming

In [12]:
#chain
response=chain.stream([{'pet':'dog'},{'pet':'dragon'}])

In [13]:
for i in response:
    print(i.content, end='')

provide consistent rules, use positive reinforcement, start training at a young age, socialize your pets, use simple commands, reward good behavior, ignore bad behavior, provide mental stimulation, provide physical exercise, be patient, hire a professional trainer if needed, create a safe environment, use a leash for control, for the dragon - establish dominance, for the dragon - use fireproof equipment, for the dragon - train in a large, open space, for the dragon - ensure proper diet to maintain health and

#### Runnable and RunnableSequence Classes

In [14]:
chat_template=ChatPromptTemplate.from_messages([
                                ('human',"Suggest me how to train my {pet}?\n"+list_instructions)])

chain=chat_template | chat

In [15]:
#runnable sequence
type(chain)

langchain_core.runnables.base.RunnableSequence

#### Piping chains and RunnablePassThrough Class

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [17]:
RunnablePassthrough().invoke('hi')

'hi'

In [18]:
RunnablePassthrough().invoke([1,2,3])

[1, 2, 3]

In [19]:
chat_template_tools=ChatPromptTemplate.from_template('''
What are the five tools a {job_title} needs?
answer only by listing the tools.
''')

chat_Template_strategy=ChatPromptTemplate.from_template('''
Consider the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

In [20]:
string_parser=StrOutputParser()

In [21]:
chain_tools = chat_template_tools | chat | string_parser
chain_strategy =chat_Template_strategy | chat | string_parser

In [22]:
print(chain_tools.invoke({'job_title':'Python'}))

1. Integrated Development Environment (IDE)
2. Text Editor
3. Python Libraries
4. Debugging Tools
5. Version Control System (VCS)


In [23]:
#chain_strategy needs dictionary to pass as input but we have generated string, lets now use RunnablePassthrough
chain_tools = chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()}
chain_strategy =chat_Template_strategy | chat | string_parser

In [24]:
#great we are getting as dictionary
print(chain_tools.invoke({'job_title':'Python'}))

{'tools': '1. Integrated Development Environment (IDE)\n2. Text Editor\n3. Python Libraries\n4. Debugging Tools\n5. Version Control System (like Git)'}


In [25]:
chain_combined = chain_tools | chain_strategy

In [26]:
print(chain_combined.invoke({'job_title':'Python'}))

1. Integrated Development Environment (IDE): Start by understanding the basic functionalities of the IDE, such as creating a new project, writing and running code, and using the built-in terminal. Gradually move on to more advanced features like debugging, version control, and refactoring. Practice using the IDE by working on small projects. 

2. Text Editor: Familiarize yourself with the basic features of the text editor, such as creating, saving, and opening files. Learn about the different syntax highlighting


#### Graphing runnables

In [27]:
chain_long=(chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()}
            | chat_Template_strategy | chat | string_parser)

In [30]:
chain_long.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       

#### RunnableParallel

In [35]:
from langchain_core.runnables import RunnableParallel

In [31]:
chat_template_books=ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming_language} books.
    Answer only by listing the books.
    '''
)

chat_template_projects=ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming_language} projects suitable for intermediate-level programmers.
    Answer only by listing the projects.
    '''
)

In [33]:
string_parser=StrOutputParser()

In [34]:
#creting both the chains
chain_books= chat_template_books|chat|string_parser
chain_projects= chat_template_projects|chat|string_parser

In [36]:
chain_parallel=RunnableParallel({'books':chain_books,'projects':chain_projects})

In [37]:
chain_parallel.invoke({'programming_language':'Rust'})

{'books': '1. "Programming Rust: Fast, Safe Systems Development" by Jim Blandy and Jason Orendorff\n2. "Rust in Action" by Tim McNamara\n3. "The Rust Programming Language" by Steve Klabnik and Carol Nichols',
 'projects': "1. Building a Web Server: This project involves creating a multi-threaded web server using Rust's standard library. It will help you understand networking concepts, error handling, and concurrency in Rust.\n\n2. Creating a Command Line Tool: This project could involve building a command-line tool like 'grep' or a simple shell. It will help you understand file I/O, regular expressions, and command-line arguments in Rust.\n\n3. Developing a Game with Piston: Piston is a user-friendly game"}

In [38]:
chain_parallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [39]:
chat_template_time=ChatPromptTemplate.from_template('''
I'm an intermediate level developer

consider the following literatute:
{books}

also, consider the following projects:
{projects}

Roughly how much time would it take me to complete the literature and the projects?
''')

In [41]:
chain_time=(RunnableParallel({'books':chain_books,'projects':chain_projects})
           |chat_template_time
           |chat
           |string_parser)

In [42]:
chain_time.invoke({'programming_language':'Rust'})

'The time it takes to complete the literature and the projects can vary greatly depending on several factors such as your current familiarity with Rust, your speed of reading and comprehension, the complexity of the projects, and the amount of time you can dedicate to studying and coding each day. \n\nHowever, as a rough estimate:\n\n1. "Programming Rust: Fast, Safe Systems Development" - This book is around 550 pages. If you read and comprehend around 20 pages a day, it would take approximately 27-28 days.\n\n2. "Rust in Action" - This book is around 450 pages. Using the same pace, it would take approximately 22-23 days.\n\n3. "The Rust Programming Language" - This book is around 280 pages. At the same pace, it would take approximately 14 days.\n\nFor the projects:\n\n1. Building a Web Server - Depending on the complexity and your familiarity with the concepts, this could take anywhere from a week to a month.\n\n2. Command Line Tool - This could take a week or two, depending on your f

In [43]:
chain_time.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

#### RunnableLambda

In [48]:
from langchain_core.runnables import RunnableLambda

In [44]:
find_sum = lambda x: sum(x)

In [45]:
find_square=lambda x: x**2

In [49]:
runnable_sum=RunnableLambda(lambda x: sum(x))

In [50]:
runnable_sum.invoke([1,2,5])

8

In [51]:
runnable_square=RunnableLambda(lambda x: x**2)

In [52]:
chain=runnable_sum | runnable_square

In [53]:
chain.invoke([1,2,5])

64

In [54]:
chain.get_graph().print_ascii()

+-------------+  
| LambdaInput |  
+-------------+  
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
+--------------+ 
| LambdaOutput | 
+--------------+ 


In [55]:
#alternative way - lnagchain decorators

def find_Sum(x):
    return sum(x)

def find_square(x):
    return x**2

chain1 = RunnableLambda(find_Sum)|RunnableLambda(find_square)

In [56]:
chain1.invoke([1,2,5])

64

In [60]:
from langchain_core.runnables import chain

@chain
def runnable_Sum(x):
    return sum(x)

@chain
def runnable_square(x):
    return x**2

In [61]:
chain2=runnable_sum | runnable_square

In [62]:
chain2.invoke([1,2,5])

64

#### Adding memory to the chain

In [70]:
from langchain.memory import ConversationSummaryMemory
from langchain_core.prompts import PromptTemplate

TEMPLATE='''
The following is a friendly conversation

current conversation:
{message_log}

Human:
{question}

AI:
'''
prompt_template=PromptTemplate.from_template(template=TEMPLATE)

In [71]:
chain = prompt_template | chat | StrOutputParser()

In [72]:
chain.invoke({'message_log':'',
             'question':"Tell me an intresting fact, probably that i don't know about?"})

'Did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body.'

In [73]:
chain.invoke({'message_log':'AI provided fact about octopus that ctopuses have three hearts. Two pump blood to the gills, while the third pumps it to the rest of the body.',
             'question':"Wow, how many variants are there?"})

'There are about 300 recognized species of octopus.'

In [141]:
chat_memory=ConversationSummaryMemory(llm=ChatOpenAI(),memory_key='message_log')

In [124]:
chat_memory.load_memory_variables({})

{'message_log': ''}

In [133]:
question= "Tell me an intresting fact, probably that i don't know about?"

In [134]:
dictionary_output=RunnablePassthrough.assign(
    message_log=RunnableLambda(chat_memory.load_memory_variables)|
    RunnableLambda(itemgetter('message_log'))).invoke({'question':question})

In [135]:
prompt_value=prompt_template.invoke(dictionary_output)

In [136]:
ai_message_output=chat.invoke(prompt_value)

In [137]:
response=StrOutputParser().invoke(ai_message_output)

In [138]:
chat_memory.save_context(inputs={'inputs':question},outputs={'outputs':response})

In [139]:
chat_memory.load_memory_variables({})

{'message_log': 'The human asks the AI to share an interesting fact. The AI reveals that octopuses have three hearts, two to pump blood to the gills and one to pump it to the rest of the body. The human expresses interest in different kinds of octopuses, and the AI explains that there are around 300 recognized species with unique characteristics and behaviors.'}

In [140]:
response

'There are around 300 recognized species of octopus. Some of the most well-known include the Common Octopus, the Giant Pacific Octopus, the Blue-Ringed Octopus, the Dumbo Octopus, and the Mimic Octopus. Each species has its own unique characteristics and behaviors.'

In [143]:
#lets create a chain

chain1=(RunnablePassthrough.assign(
    message_log=RunnableLambda(chat_memory.load_memory_variables)|
    RunnableLambda(itemgetter('message_log')))
        |prompt_template
        |chat
        |StrOutputParser()
       )
question="Tell me an intresting fact, probably that i don't know about?"
response=chain1.invoke({'question':question})
chat_memory.save_context(inputs={'inputs':question},outputs={'outputs':response})
response

'Did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body.'

In [144]:
question="Wow, what are the different kinds of them?"
response=chain1.invoke({'question':question})
chat_memory.save_context(inputs={'inputs':question},outputs={'outputs':response})
response

'There are around 300 recognized species of octopuses. Some of the most well-known include the Common Octopus, Giant Pacific Octopus, Blue-Ringed Octopus, Dumbo Octopus, and the Mimic Octopus. Each species has its own unique characteristics and behaviors.'

In [145]:
question="Can i eat it?"
response=chain1.invoke({'question':question})
chat_memory.save_context(inputs={'inputs':question},outputs={'outputs':response})
response

"Yes, octopus is commonly consumed in many cultures and is a popular part of Mediterranean and Asian cuisines. However, it's important to note that some species, like the Blue-Ringed Octopus, are venomous and not safe to eat. Always ensure that any seafood you consume is properly prepared and safe to eat."

In [148]:
#lets use decarators
from langchain_core.runnables import chain
@chain
def memory_chain(question):
    chain1=(RunnablePassthrough.assign(
        message_log=RunnableLambda(chat_memory.load_memory_variables)|
        RunnableLambda(itemgetter('message_log')))
            |prompt_template
            |chat
            |StrOutputParser()
           )
    response=chain1.invoke({'question':question})
    chat_memory.save_context(inputs={'inputs':question},outputs={'outputs':response})
    return response

In [150]:
memory_chain.invoke('Tell me an interesting fact about india?')

"India is the world's largest producer and consumer of spices. The country produces about 70% of the world's spices."

In [151]:
memory_chain.invoke('Name some of them?')

'Sure, some of the spices that India produces include turmeric, cardamom, cumin, coriander, fenugreek, ginger, clove, black pepper, and red chili pepper.'